## TO DO:
- [-] Take rough number of images to be downloaded as input. (Rough because few images will be corrupt and deleted later on.) - MAY NOT NEED THIS, SEE BELOW. We could also gather URLS and then select the number of URLs to scrape. But this would cap the number of images at 100, since the script scrapes 100 URLS by default, not counting the few corrupt images that we'd lose.
- [x] The make_image_dataset function should be able to take Lists as input. When taking list inputs, need to create folders within destination folder to segregate the images.

## Getting more data:
1. Tagged Anime Illustrations: <br>
https://www.kaggle.com/mylesoneill/tagged-anime-illustrations/home - Will need to clone this directly into GCP. 36 GB is too much for my pleb laptop.

2. Anime Faces : <br>
http://www.nurs.or.jp/~nagadomi/animeface-character-dataset/

3. Check out the links above they lead to tabular anime data as well.

### We can this the dataset gathered for Anime Identification. That is, if the above falls short.

## Lesson learnt:
## Save all FAST AI notebooks that you edit to your local storage.

In [1]:
#!pip install pyvirtualdisplay
#!pip install xvfbwrapper
#!sudo apt-get install xvfb

In [14]:
from imutils import paths
import argparse
import requests
import cv2
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
import glob
import time
from pyvirtualdisplay import Display
from threading import *
from multiprocessing import Pool

In [15]:
Anime_Faces_List_Fixed = ["Cowboy Bebop","Steins Gate","Maison Ikkoku","Baccano","Mushishi","The Rose of Versailles","Berserk","Now and Then Here and There","Legend of the Galactic Heroes","Usagi Drop","Rurouni Kenshin Wandering Samurai","Guardian of the Sacred Spirit","Planetes","Great Teacher Onizuka","The Irresponsible Captain Tylor","Lupin the 3rd","Death Note","Monster","Elfen Lied","Hunter x Hunter","Neon Genesis Evangelion","City Hunter","Ghost in the Shell Stand Alone Complex","Code Geass Lelouch of the Rebellion","Tomorrows Joe","Black Lagoon","Darker Than Black: Gemini of the Meteor","Yu Yu Hakusho Ghost Files","Ranma","Samurai Champloo","Gungrave","Tengen toppa gurren lagann","Claymore","Fullmetal Alchemist","Hellsing Ultimate","Kimagure Orange Road","Future Diary","Eureka Seven","Katanagatari","Princess Tutu","Tenchi Muyo","Spice and Wolf","Super Dimensional Fortress Macross","Bakemonogatari","School Rumble","Hajime no ippo","Revolutionary Girl Utena","Natsumes Book of Friends","Inuyasha","The Twelve Kingdoms","Kaiba","Escaflowne","Koi kaze","When They Cry","Ergo Proxy","Durarara","Mobile Suit Gundam","FLCL","Future Boy Conan","The Count of Monte Cristo","Welcome to the NHK","Nobodys Boy Remi","The Slayers","Gambling Apocalypse Kaiji","Kimi ni todoke","Serial Experiments Lain","Ouran High School Host Club","Les Miserables Shoujo Cosette","Michiko and Hatchin","Romeos Blue Skies","Mononoke","Phantom Requiem for the Phantom","Rainbow Nishakubou no shichinin","The Mysterious Cities of Gold","Sayonara Zetsubou Sensei","Anne of Green Gables anime","Space Pirate Captain Harlock","Touch","Chobits","Treasure Island","Urusei yatsura","Wolfs Rain","Fist of the North Star","Seitokai yakuindomo","Ef A Tale of Memories","Trigun","Haibane renmei","Toradora","Kaleido Star","Blood plus","Golden Boy","JoJos Bizarre Adventure","The Melancholy of Haruhi Suzumiya","XXXHOLiC","Nodame Cantabile","Crest of the Stars","Clannad","Texhnolyze","Yojohan shinwa taikei"]

In [16]:
images_to_download_list = ["naruto", "cowboy bebop"]

In [17]:
def make_image_dataset(image_name, dest_folder = r"/home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images/", downloads_folder= r"/home/jupyter/Downloads/"):    
    #images_to_download = input("Which images do you want to download? ")
    #dest_folder = input("Where do you want the downloaded images? ")
    #downloads_folder = input("Enter the path to your downloads folder. ")
    
    os.mkdir(os.path.sep.join([dest_folder, image_name]))
    dest_folder_new = os.path.sep.join([dest_folder, image_name])
        
    options = webdriver.ChromeOptions()
    #options.add_argument('headless')
    #options.add_argument('window-size=1200x600')  
    display = Display(visible=0, size=(1200, 600))
    display.start()
        
    driver = webdriver.Chrome(executable_path = "/home/jupyter/chromedriver.exe", options=options)
    
    driver.get("https://www.google.com/search?tbm=isch&q=" + image_name)
    
    urls = driver.execute_script(r'''urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
    window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));''')
    time.sleep(4)
    
    driver.quit()
    display.stop()
    
    #list_of_files = glob.glob(r"C:\Users\kmoudgalya\Downloads\*") # * means all if need specific format then *.csv
    list_of_files = glob.glob(os.path.sep.join([downloads_folder, "/*"]))
    latest_file = max(list_of_files, key=lambda x: os.stat(x).st_mtime)
    
    rows = open(latest_file).read().strip().split("\n")
    total = 0
    
    
    # loop the URLs
    for url in rows:
        try:
            # try to download the image
            r = requests.get(url, timeout=10)

            # save the image to disk
            p = os.path.sep.join([dest_folder_new, "{}.jpg".format(str(total).zfill(8))])
            f = open(p, "wb")
            f.write(r.content)
            f.close()

            # update the counter
            print("[INFO] downloaded: {}".format(p))
            total += 1

        # handle if any exceptions are thrown during the download process
        except:
            print("[INFO] error downloading {}...skipping".format(p))
    
    
    # loop over the image paths we just downloaded
    for imagePath in paths.list_images(dest_folder_new):
        # initialize if the image should be deleted or not
        delete = False

        # try to load the image
        try:
            image = cv2.imread(imagePath)

            # if the image is `None` then we could not properly load it
            # from disk, so delete it
            if image is None:
                delete = True

        # if OpenCV cannot load the image then the image is likely
        # corrupt so we should delete it
        except:
            print("Except")
            delete = True

        # check to see if the image should be deleted
        if delete:
            print("[INFO] deleting {}".format(imagePath))
            os.remove(imagePath)
            
    print("Images downloaded. Let the Machine Learning begin.")

In [20]:
start_time = time.time()
make_image_dataset("cowboy bepop")
print('Entire job took:',time.time() - start_time)

[INFO] downloaded: ./anime_faces/cowboy bepop/00000000.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000001.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000002.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000003.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000004.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000005.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000006.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000007.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000008.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000009.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000010.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000011.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000012.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000013.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000014.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000015.jpg
[INFO] downloaded: ./anime_faces/cowboy bepop/00000016.j

### Time taken by function:
#### Naruto images download      : 32.81 s <br>
#### Cowboy Bepop images download: 40.70 s <br>
#### Total time                  : 73.51 s <br>

# Multithreading:

In [18]:
!rm -r /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images
!mkdir /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images

In [19]:
start_time = time.time()
threadlist = []
for j in Anime_Faces_List_Fixed:
    print(j)
    t = Thread(target=make_image_dataset, args=(j,))
    
    t.start()
    threadlist.append(t)
    
for b in threadlist:
   b.join()
    
print('Entire job took:',time.time() - start_time)

Cowboy Bebop
Steins Gate
Maison Ikkoku
Baccano
Mushishi
The Rose of Versailles
Berserk
Now and Then Here and There
Legend of the Galactic Heroes
Usagi Drop
Rurouni Kenshin Wandering Samurai
Guardian of the Sacred Spirit
Planetes
Great Teacher Onizuka
The Irresponsible Captain Tylor
Lupin the 3rd
Death Note
Monster
Elfen Lied
Hunter x Hunter
Neon Genesis Evangelion
City Hunter
Ghost in the Shell Stand Alone Complex
Code Geass Lelouch of the Rebellion
Tomorrows Joe
Black Lagoon
Darker Than Black: Gemini of the Meteor
Yu Yu Hakusho Ghost Files
Ranma
Samurai Champloo
Gungrave
Tengen toppa gurren lagann
Claymore
Fullmetal Alchemist
Hellsing Ultimate
Kimagure Orange Road
Future Diary
Eureka Seven
Katanagatari
Princess Tutu
Tenchi Muyo
Spice and Wolf
Super Dimensional Fortress Macross
Bakemonogatari
School Rumble
Hajime no ippo
Revolutionary Girl Utena
Natsumes Book of Friends
Inuyasha
The Twelve Kingdoms
Kaiba
Escaflowne
Koi kaze
When They Cry
Ergo Proxy
Durarara
Mobile Suit Gundam
FLCL
Futu

Exception in thread Thread-39:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-17-4d19ad8b14f7>", line 15, in make_image_dataset
    driver = webdriver.Chrome(executable_path = "/home/jupyter/chromedriver.exe", options=options)
  File "/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/chrome/webdriver.py", line 81, in __init__
    desired_capabilities=desired_capabilities)
  File "/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 157, in __init__
    self.start_session(capabilities, browser_profile)
  File "/opt/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 252, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
  File "/opt/anaconda3/lib/python3.7/site-

[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000000.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000001.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000002.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000003.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000004.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000005.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000006.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Eureka Seven/00000000.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000007.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000008.jpg
[INFO

Exception in thread Thread-63:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/anaconda3/lib/python3.7/http/client.py", line 1321, in getresponse
    response.begin()
  File "/opt/anaconda3/lib/python3.7/http/client.py", line 296, in begin
    version, status, reason = self._read_status()
  File "/opt/anaconda3/lib/python3.7/http/client.py", line 265, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception,

[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Planetes/00000019.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Now and Then Here and There/00000019.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Code Geass Lelouch of the Rebellion/00000019.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//JoJos Bizarre Adventure/00000019.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Maison Ikkoku/00000019.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Blood plus/00000019.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Bakemonogatari/00000019.jpg[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Chobits/00000019.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//The Mysterious Cities of Gold/00000019.jpg
[INFO] 

Exception in thread Thread-103:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/anaconda3/lib/python3.7/http/client.py", line 1321, in getresponse
    response.begin()
  File "/opt/anaconda3/lib/python3.7/http/client.py", line 296, in begin
    version, status, reason = self._read_status()
  File "/opt/anaconda3/lib/python3.7/http/client.py", line 265, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception

KeyboardInterrupt: 

[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Eureka Seven/00000028.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Eureka Seven/00000029.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Berserk/00000037.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Now and Then Here and There/00000020.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Code Geass Lelouch of the Rebellion/00000020.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Planetes/00000020.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//Blood plus/00000020.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//The Mysterious Cities of Gold/00000020.jpg
[INFO] downloaded: /home/jupyter/Anime_Project/data_v3/Anime_Recogntn_Images//JoJos Bizarre Adventure/00000020.jpg
[INFO] do

### 24.05s (multi-threaded) vs 73.51s for the script to download images. I'd say this is a no brainer. 3x Improvement ftw! This is with 2 separate image targets (naruto and cowboy bepop).

# Multiprocessing

In [12]:
!rm -r anime_faces
!mkdir anime_faces

In [13]:
start_time = time.time()
x = Pool(20)
records = x.map(make_image_dataset,images_to_download_list)
x.terminate()
x.join()
print('Entire job took:',time.time() - start_time)

[INFO] downloaded: ./anime_faces/cowboy bebop/00000000.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000001.jpg
[INFO] downloaded: ./anime_faces/naruto/00000000.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000002.jpg
[INFO] downloaded: ./anime_faces/naruto/00000001.jpg
[INFO] downloaded: ./anime_faces/naruto/00000002.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000003.jpg
[INFO] downloaded: ./anime_faces/naruto/00000003.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000004.jpg
[INFO] downloaded: ./anime_faces/naruto/00000004.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000005.jpg
[INFO] downloaded: ./anime_faces/naruto/00000005.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000006.jpg
[INFO] downloaded: ./anime_faces/naruto/00000006.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000007.jpg
[INFO] downloaded: ./anime_faces/naruto/00000007.jpg
[INFO] downloaded: ./anime_faces/cowboy bebop/00000008.jpg
[INFO] downloaded: ./anime_faces/naruto/00000

### When Pool = 5, 31s
### When Pool = 10, 27s 
### When Pool = 20, 26s

### 100x100 images (10000) take around 6-7 hours to be downloaded. Without Multithreading ofc.